In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime

%load_ext autoreload
%autoreload 2
import my_functions as mf

import warnings
warnings.filterwarnings("ignore")




In [ ]:
# Opciones de visualización de Dataframes
pd.set_option('display.max_columns', None)
'''
pd.set_option('display.max_rows', None)
pd.reset_option('display.max_rows')
pd.reset_option('display') 
'''

## Carga de datos

Para empzar se realizará la carga de datos del archivo 'homicidios.xlsx' que contiene 2 hojas de datos propiamente dichos, 'HECHOS' Y 'VICTIMAS', además de otras pestañas que contienen información descriptiva sobre los datos.

Se realizará la carga de las 2 hojas de datos en los dataframes df_hechos y df_victimas respectivamente, para luego proceder con la exploración de los mismos.

In [ ]:
df_hechos = pd.read_excel('Datasets/homicidios.xlsx', sheet_name='HECHOS')

In [ ]:
df_victimas = pd.read_excel('Datasets/homicidios.xlsx', sheet_name='VICTIMAS')

In [ ]:
# Se verifica la carga
df_hechos.info()

In [ ]:
# Se verifica la carga
df_victimas.info()

Teniendo cargados ambos dataframe procederemos con la exploración de cada uno, comenzando por el dataframe referido a hechos.

## ETL 'HECHOS' (df_hechos)

En primer lugar, se explorarán los campos, sus tipos de datos, datos nulos, valores duplicados y únicos, para obtener un panorama general de la información.

In [ ]:
# Se aplica función personalizada para describir información de relevancia
mf.describir_df(df_hechos)

In [ ]:
# Se visualizan algunos registros
df_hechos.head(3)

### Observaciones iniciales

En total cuenta con **696 registros con 21 campos**:
'ID', 'N_VICTIMAS', 'FECHA', 'AAAA', 'MM', 'DD', 'HORA', 'HH', 'LUGAR_DEL_HECHO', 'TIPO_DE_CALLE', 'Calle', 'Altura', 'Cruce', 'Dirección Normalizada', 'COMUNA', 'XY (CABA)', 'pos x', 'pos y', 'PARTICIPANTES', 'VICTIMA', 'ACUSADO'.

Entre los mismos encontramos variables temporales, espaciales y de participantes asociadas al hecho:
- **Identificador único** para cada registro: campo 'ID'. Compuesto por el año del hecho y un número identificatorio ("aaaa-numero").
- Información referida a los **participantes** involucrados en el siniestro: campos 'N_VICTIMAS' (refiere a cantidad de víctimas fatales producto del hecho), 'PARTICIPANTES', 'VICTIMA', 'ACUSADO'.
- **Información temporal**: campos 'FECHA', 'AAAA', 'MM', 'DD', 'HORA', 'HH'.
- **Información espacial**: 'LUGAR_DEL_HECHO', 'TIPO_DE_CALLE', 'Calle', 'Altura', 'Cruce', 'Dirección Normalizada', 'COMUNA', 'XY (CABA)', 'pos x', 'pos y'.

El único campo con **valores únicos** es **'ID'**, que corresponde al identificador único de cada registro (es decir, de cada hecho).

Nos encontramos con **valores nulos** en 4 campos (**'Calle', 'Altura', 'Cruce', 'Dirección Normalizada'**), todos ellos entre lo que refiere a **información espacial**.

Por otro lado, como se explica en las [**notas de uso**](NOTAS_HOMICIDIOS_SINIESTRO_VIAL.pdf) relativas a la base de datos, en algunos casos se imputa 'SD' (sin datos) para valores faltantes.

Antes de proseguir con la exploración se estandarizarán los nombres de los campos.

In [ ]:
print("Campos originales:\n", df_hechos.columns, "\n")

# Reemplazar campos específicos
reemplazos = {
    'ID': 'Id_hecho',
    'N_VICTIMAS': 'Cantidad_victimas',
    'AAAA': 'Año',
    'MM': 'Mes',
    'DD': 'Dia',
    'HH': 'Franja_horaria',
    'LUGAR_DEL_HECHO': 'Lugar_hecho',
    'TIPO_DE_CALLE': 'Tipo_calle',
    'Dirección Normalizada': 'Direccion_normalizada',
    'pos x' : 'Pos_x',
    'pos y' : 'Pos_y'
}

df_hechos.columns = df_hechos.columns.to_series().replace(reemplazos).str.capitalize()
# Renombrar 
df_hechos.rename(columns={'Xy (caba)': 'XY_CABA'}, inplace=True)

print("Campos modificados:\n", df_hechos.columns)


Tras estas modificaciones procederemos con exploración de valores nulos para determinar cómo tratarlos.

### Datos nulos o sin datos ('SD')

Cómo se mencionó, tenemos valores nulos en 4 campos. 

- Calle:	                1	registro (0.14%)	
- Altura:	                567	registros (81.47%)	
- Cruce:	                171	registros (24.57%)
- Dirección Normalizada:	8	registros (1.15%)

Además de estas variables, contamos con otras referidas a información espacial del incidente que no contienen valores nulos y pueden ser suficientes para nuestro análisis es términos de la dimensión geográfica.

Por otro lado, tenemos los valores registrados con 'SD' (Sin Datos).

Antes de abordar los valore nulos se observan estos registros.

In [ ]:
# Se buscan valores 'SD' en el dataframe
valor = ['SD']
mf.buscar_valor(df_hechos, valor)

En total 28 filas presentan al menos un campo sin datos (sin contabilizar valores nulos), con 35 apariciones en total.

Particularmente, se encuentran en los campos 'Hora': 1, 'Franja_horaria': 1, 'Lugar_hecho': 1, 'Victima': 9 y 'Acusado': 23.

Ningún campo presenta simultáneamente valores con 'SD' y nulos.

Por el momento, los valores con 'SD' se imputaran como nulos y luego se definirá como tratarlos según sea el caso.

In [ ]:
mf.imputar_valores(df_hechos, valor_a_reemplazar = 'SD', valor_a_imputar = None, campos=None)

Se puede observar que ahora se suman estos valores a los nulos que ya teníamos.

In [ ]:
mf.buscar_valor(df_hechos, valores = None, campos=None)

Más adelante exploraremos estos datos faltantes para definir cómo tratarlos y la información que será objetivo de análisis.

### Duplicados

Sabiendo que para 'Id_hecho' no contamos con valores duplicados, se tiene la certeza de que todos los registros cuentan con su identificador único.

In [ ]:
# Se trae información del campo 'Id_hecho'
mf.describir_df(df_hechos, campos_incluir=['Id_hecho'])

Se verifica si existen registros duplicados excluyendo dicho campo, para descartar duplicados con un identificador distinto.

In [ ]:
# Duplicados en todo el DataFrame excluyendo columna 'Id_hecho'
mf.duplicados(df_hechos, campos_excluir=['Id_hecho'])

Ningún registro contiene información duplicada en el resto de los campos.

A continuación se exploran las variables asociadas según sean estas temporales, espaciales y referidas a los participantes, de forma tal de profundizar en el contenido de las mismas y definir cuáles serán utilizadas para el análisis, identificar información relevante o redundante si la hubiese.

## Variables sobre participantes ('Cantidad_victimas', 'Participantes',	'Victima',	'Acusado')

Se agrupan estas variables y se traen algunos registros.

In [ ]:
# Se genera un dataframe con la información de variables 
campos_participantes = ['Cantidad_victimas', 'Participantes',	'Victima',	'Acusado']
df_participantes = df_hechos[campos_participantes]
mf.registros_al_azar(df_participantes, 3)

Se observan valores únicos para cada campo.

In [ ]:
# Se observan valores únicos para cada campo
mf.valores_unicos(df_participantes, campos_participantes)

Algunas observaciones:
- La cantidad de víctimas es entre 1 y 3 personas.
- En el campo 'Victima' se encuentran valores 'OBJETO FIJO' y 'PEATON_MOTO', que no se encuentran como valores posibles en las notas de uso correspondientes al dataset original provisto.
- En el campo 'Participantes' se observan múltiples combinaciones, tomando valores de los campos anteriormente mencionados. En algunos casos se observan valores 'SD' combinados con otros, o bien 'SD-SD'.
- 'Victima' y 'Acusado' presentan valores nulos.

Se observarán los registros donde se encontraron datos no incluídos en las notas de uso.

In [ ]:
# Se traer registros donde la columna 'Victima' es igual a 'OBJETO FIJO' o 'PEATON_MOTO'
df_participantes[df_participantes['Victima'].isin(['OBJETO FIJO', 'PEATON_MOTO'])]

Se observa que en ambos casos hubo 2 víctimas. Posiblemente el segundo registro refiera a una víctima peatón y otra en moto. Sin embargo, al no poder precisar el vehículo de las víctimas, se modificarán estos valores por 'OTRO', siguiendo las indicaciones de las notas de uso.

In [ ]:
# Reemplazar 'AUTO-OBJETO FIJO' por 'OTRO- AUTO' en el campo 'Participantes'
df_participantes.at[230, 'Participantes'] = 'OTRO-AUTO'

# Reemplazar 'OBJETO FIJO' por 'OTRO' en el campo 'Victima'
df_participantes.at[230, 'Victima'] = 'OTRO'

df_participantes.iloc[230]

In [ ]:
# Reemplazar 'AUTO-OBJETO FIJO' por 'AUTO-OTRO' en el campo 'Participantes'
df_participantes.at[583, 'Participantes'] = 'OTRO-MOTO'

# Reemplazar 'OBJETO FIJO' por 'OTRO' en el campo 'Victima'
df_participantes.at[583, 'Victima'] = 'OTRO'

df_participantes.iloc[583]

Se verifican los cambios obsevando los valores únicos de ambos campos.

In [ ]:
mf.valores_unicos(df_participantes, ['Victima', 'Acusado'])

Los valores nulos de estos 2 campos serán imputados con el valor más frecuente de cada uno, visto y considerando que no representan una cantidad significativa de los datos.

In [ ]:
# Se busca la moda para cada campo
print(mf.buscar_moda(df_participantes, 'Victima'))
print(mf.buscar_moda(df_participantes, 'Acusado'))

In [ ]:
# Se imputa en 'Victima' el valor 'MOTO'
mf.imputar_valores(df_participantes, valor_a_reemplazar=None, valor_a_imputar='MOTO', campos=['Victima'])
mf.valores_unicos(df_participantes, ['Victima'])

In [ ]:
# Se imputan en 'Acusado' el valor 'AUTO'
mf.imputar_valores(df_participantes, valor_a_reemplazar=None, valor_a_imputar='AUTO', campos=['Acusado'])
mf.valores_unicos(df_participantes, ['Acusado'])

Habiendo realizado estas modificaciones, se determina que la variable 'Participantes' no aporta información valiosa, ya que contiene los valores de 'Victima' y 'Acusado', motivo por el que se descarta.

Se procede a actualizar los datos con estas modificaciones en el dataframe original.

In [ ]:
# Copiar la información de df_temporal a df_hechos
df_hechos.update(df_participantes['Victima'])
df_hechos.update(df_participantes['Acusado'])

# Eliminar las columnas especificadas
df_hechos = df_hechos.drop(['Participantes'], axis=1)

In [ ]:
# Se verifican las modificaciones
mf.describir_df(df_hechos)

## Variables temporales ('Fecha', 'Año', 'Mes', 'Dia', 'Hora', 'Franja_horaria')

Se agrupan estas variables y se traen reiterados registros al azar para ver la relación entre cada variable.

In [ ]:
# Se genera un dataframe con la información de variables temporales
campos_temporal = ['Fecha', 'Año', 'Mes', 'Dia', 'Hora', 'Franja_horaria']
df_temporal = df_hechos[campos_temporal]

In [ ]:
# Se traen registros al azar para visualizar la información disponible en distintos registros
mf.registros_al_azar(df_temporal, 10)

Tras explorar varios grupos de registros al azar queda claro qué el campo 'Fecha' combina la información de 'Año', 'Mes', 'Dia', 'Hora' se muestra en formato hh:mm:ss y 'Franja_horaria' refiere a la hora como número entero.

Sin embargo, se observan algunos valores ingresados incorrectamente, combinando en un campo información variada (por ejemplo, fecha y hora en un mismo valor).

En 'Hora' y 'Franja_horaria' encontramos múltiples tipos de datos.

Para continuar, verificaremos valores de cada campo invividualmente.

In [ ]:
pd.set_option('display.max_colwidth', None)
mf.tipo_datos(df_temporal)

En la variable 'Hora', se exploran específicamente los valores tipo 'str' (cadena de texto) y luego se comparan con otros valores.

In [ ]:
# Filtrar y mostrar solo los campos de tipo str en la columna 'Hora'
mf.registros_al_azar((df_temporal['Hora'][df_temporal['Hora'].apply(lambda x: isinstance(x, str))]), 5)

In [ ]:
mf.registros_al_azar(df_temporal['Hora'], 5)

Se observa en todos los casos un formato "hh:mm:ss".

Se modifica el tipo de datos de todos los valores a 'datetime.time'.

In [ ]:
# Aplica la función a la columna 'Hora', excluyendo valores None
df_temporal['Hora'] = df_temporal['Hora'].apply(mf.convertir_a_datetime).dropna()

mf.tipo_datos(df_temporal, ['Hora'])

In [ ]:
# Se identifica registro con valor nulo
df_temporal['Hora'][df_temporal['Hora'].apply(lambda x: isinstance(x, float))]

Un registro aún es nulo.

In [ ]:
df_temporal.iloc[518]

Se observa que 'Franja_horaria' también es nula en el registro correspondiente.

Considerando que se trata solamente de un registro se determina imputar la hora del hecho con el valor más frecuente.

In [ ]:
hora_mas_frecuente = df_temporal['Hora'].mode()[0]
hora_mas_frecuente

Se imputa 9 horas como el valor moda.

In [ ]:
# Imputar valores en el registro 518
df_temporal.at[518, 'Hora'] = hora_mas_frecuente
df_temporal.at[518, 'Franja_horaria'] = int(hora_mas_frecuente.hour)
    
# Verificar los cambios
df_temporal.iloc[518]

Se observa que los valores han sido modificados aplicando la moda de la hora.

Verificamos el cambio de tipo de datos, corroborando que 'Hora' y 'Franja_horaria' cuentan con el tipo de datos correcto para todos los registros.

In [ ]:
mf.tipo_datos(df_temporal)

Por último chequeamos valores mínimos y máximos de los campos para asegurar que no haya información errónea.

In [ ]:
mf.max_min_campos(df_temporal, campos_temporal)

Se verifican los valores, pudiendo observar que hay hechos registrados desde el 1 de enero de 2016 hasta el 30 de diciembre de 2021.

Habiendo realizado todas las modificaciones pertinentes sobre este conjunto de datos, se procede a volcar la información en el conjunto original.

In [ ]:
# Copiar la información de df_temporal a df_hechos
df_hechos.update(df_temporal[campos_temporal])

mf.describir_df(df_hechos, campos_incluir = campos_temporal)

## Variables espaciales 

A continuación se exploran las variables espaciales.

Se cuenta con detalles referidos a dónde ocurrió el hecho. Particularmente se pueden observar variables referidas especicamente al tipo de arteria (calle, avenida, autopista, General Paz) y detalles de la misma, y otras variables de geolocalización (latitud, longitud) que permiten una ubicación precisa en el mapa.

### Detalles sobre arteria donde ocurrió el hecho 

Estos campos refieren a la ubicación específica dónde ocurrió el hecho en la Ciudad de Buenos Aires y detalles al respecto ('Lugar_hecho', 'Tipo_calle', 'Calle', 'Altura', 'Cruce', 'Direccion_normalizada', 'Comuna').

In [ ]:
# Se genera un dataframe con la información de variables 
campos_calle = ['Lugar_hecho', 'Tipo_calle', 'Calle', 'Altura', 'Cruce', 'Direccion_normalizada', 'Comuna']
df_calle = df_hechos[campos_calle]

mf.describir_df(df_calle)

In [ ]:
mf.tipo_datos(df_calle)

En el campo 'Altura' se encuentra un total de 567 registros nulos (más del 81%).

Dado que contamos con varios campos que pueden aportar información complementaria, se revisarán aquellos referidos a la calle ('Lugar_hecho', 'Tipo_calle', 'Calle', 'Altura', 'Cruce',
       'Direccion_normalizada').

In [ ]:
# Se traen registros al azar para visualizar la información disponible en distintos registros
mf.registros_al_azar(df_calle, 3)

Tras explorar varios registros se oberva:
- La información de 'Lugar_hecho' y de 'Direccion_normalizada 'combinan información de 'Calle', 'Altura' y 'Cruce', conteniendo prácticamente los mismo datos.
- Donde 'Altura' contiene valor nulo, implica que el hecho ocurrió en un cruce.
- Inversamente, donde 'Cruce' no contiene datos, se cuenta con la altura de la calle.

A partir de estas observaciones, se concluye que la variable 'Cruce' nos puede indicar si esto ocurrió en una esquina o en algún punto de la calle/avenida/autopista que no cruce con otra, y en 'Dirección_normalizada' contamos con la información de 'Calle' y 'Altura'/'Cruce'.

Bajo estas consideraciones, se decide:
- Prescindir de los campos 'Lugar_hecho' y 'Altura' (en ambos casos la información está contenido en 'Dirección_normalizada', ademas de contar con datos de geolocalización).
- Transformar la columna 'Curce' en una variable booleana ('Si', en caso que sea un cruce, 'No' si no hay datos en el registro), también considerando que el nombre del cruce se puede encontrar en 'Dirección_normalizada'.

Se conserva el campo 'Calle', a pesar de está contenido en 'Direccion_normalizada' y ser información redundante, ya que puede aportar al análisis que se realizará luego.

Primero se exploran registros nulos de 'Direccion_normalizada'.

In [ ]:
mf.buscar_valor(df_calle, valores=None, campos = ['Direccion_normalizada'])
mf.mostrar_registros(df_calle, valor=None, campos = ['Direccion_normalizada'])

Estos registros coinciden en que todos ocurrieron en autopista, a excepción de uno que no contiene datos en los campos de calle.

Si bien la información de 'Direccion_normalizada' no resulta imprescindible, en los campos con valor nulo se tomarán los datos de 'Lugar_hecho' en reemplazo.

In [ ]:
def reemplazar_direccion(row):
    if pd.isnull(row['Direccion_normalizada']):
        return row['Lugar_hecho']
    else:
        return row['Direccion_normalizada']

# Aplicar la función a la columna 'Direccion_normalizada'
df_calle['Direccion_normalizada'] = df_calle.apply(reemplazar_direccion, axis=1)

En el registro 119, dado que no contiene valores para completar, se imputa 'SD' en los campos 'Calle' y  'Direccion_normalizada'.

En 'Cruce' se imputará el valor más frecuente una vez convertida la columna a 'SI' / 'NO'.

In [ ]:
# Reemplazar el valor en la fila específica
df_calle.at[119, 'Direccion_normalizada'] = 'SD'
df_calle.at[119, 'Calle'] = 'SD'

Se verifican las modificaciones. 

In [ ]:
mf.mostrar_registros(df_calle, valor=None, campos = ['Direccion_normalizada'])

In [ ]:
indices_a_mostrar = [38, 106, 119, 180, 181, 313, 546, 621]
registros_seleccionados = df_calle.loc[indices_a_mostrar]
registros_seleccionados

Se prosigue con la información relativa a 'Cruce', realizando la transformación mencionada arriba.

In [ ]:
# Los datos nulos de la columna 'Cruce' toman valor 'NO', y donde se registro una calle como cruce se reemplaza por 'SI'
def transformar_cruce(x):
    if pd.isna(x):
        return 'NO'
    else:
        return 'SI'

df_calle['Cruce'] = df_calle['Cruce'].apply(transformar_cruce)

In [ ]:
# Se identifica valor más frecuente para 'Cruce'
mf.buscar_moda(df_calle, campos = ['Cruce'])

In [ ]:
# Reemplazar el valor en la fila específica
df_calle.at[119, 'Cruce'] = 'SI'

In [ ]:
# Se contabilizan registros sin cruce
print(f"Cantidad de registros con 'Cruce' igual a 'NO': {len(df_calle[df_calle['Cruce'] == 'NO'])}")

Los 171 registros nulos para 'Cruce' ahora tienen como valor 'NO', indicando que el hecho no ocurrió en un cruce, a excepción del que se le imputó 'SI' por ser el valor más frecuente.

Se actualiza la información en el dataframe original, se descartan las columnas 'Lugar_hecho' y 'Altura', y se verifica la actualizacion.

In [ ]:
# Actualizar información de df_calle a df_hechos
df_hechos.update(df_calle['Cruce'])
df_hechos.update(df_calle['Direccion_normalizada'])
df_hechos.update(df_calle['Calle'])

# Eliminar las columnas especificadas
df_hechos = df_hechos.drop(['Lugar_hecho', 'Altura'], axis=1)

mf.describir_df(df_hechos)

Tras estas modificaciones ya no tenemos valores nulos.

Se observan 1 registro con los campos 'SD' que imputamos.

In [ ]:
mf.buscar_valor(df_hechos, valores = 'SD', campos=None)

### Detalles de geolocalización

Se prosigue con la información vinculada a geolocalización ('XY_CABA', 'Pos_x', 'Pos_y').

In [ ]:
# Se genera un dataframe con la información de variables espaciales
campos_geo = ['XY_CABA', 'Pos_x', 'Pos_y']
df_geo = df_hechos[campos_geo]

mf.describir_df(df_geo)

Se explora si existen valores 'SD' en estos campos y no se encuentra ninguno.

In [ ]:
# Contar valores 'SD' 
print(f"Registros 'SD':")
print(f"Columna 'XY_CABA': {df_geo['XY_CABA'].value_counts().get('SD', 0)}")
print(f"Columna 'Pos_x': {df_geo['Pos_x'].value_counts().get('SD', 0)}")
print(f"Columna 'Pos_y': {df_geo['Pos_y'].value_counts().get('SD', 0)}")

In [ ]:
# Se traen registros al azar para visualizar la información disponible en distintos registros
mf.registros_al_azar(df_geo, 10)

Se observan algunos valor 'Point (. .)' en 'XY_CABA' y '.' en 'Pos_x' y Pos_y', lo cual indica que no se cuenta con información sobre estas ubicaciones. Se reemplazan por 0.

In [ ]:
mf.buscar_valor(df_geo, valores = 'Point (. .)')
mf.buscar_valor(df_geo, valores = '.')

In [ ]:
df_geo['XY_CABA'] = df_geo['XY_CABA'].replace('Point (. .)', 0)
df_geo['Pos_x'] = df_geo['Pos_x'].replace('.', 0)
df_geo['Pos_y'] = df_geo['Pos_y'].replace('.', 0)

# Imprimir el resultado
print(f"Cantidad de valores iguales a 0 en 'XY_CABA': {df_geo['XY_CABA'].value_counts().get(0, 0)}")
print(f"Cantidad de valores iguales a 0 en 'Pos_x': {df_geo['Pos_x'].value_counts().get(0, 0)}")
print(f"Cantidad de valores iguales a 0 en 'Pos_y': {df_geo['Pos_y'].value_counts().get(0, 0)}")

In [ ]:
df_geo.loc[df_geo['XY_CABA'] == 0]

Se observan 12 registros totales que no cuentan con información de geolocalización. 

En 2 registros si bien no se cuenta con datos para 'XY_CABA', si existen datos de latitud y longitud. 


Dado que con las coordenadas de latitud y longitud es suficiente para geolocalizar los hechos, el campo 'XY_CABA' no es necesario, por lo que se lo descarta del dataset.

In [ ]:
# Actualizar información de df_calle a df_hechos
df_hechos.update(df_geo['Pos_x'])
df_hechos.update(df_geo['Pos_y'])

# Eliminar las columnas especificadas
df_hechos = df_hechos.drop(['XY_CABA'], axis=1)

# Se verifica la carga
df_hechos.head(1)

Con esto se puede dar por finalizada la revisión inicial y las transformaciones necesarias sobre los datos provistos en la hoja 'HECHOS' del archivo Excel provisto. 

Se reserva el dataframe con las transformaciones realizadas hasta el momento y se procede con la reivisión de la hoja 'VICTIMAS' del mismo archivo (nuestro datafram 'df_victimas').

## ETL 'VICTIMAS' (df_victimas)

Al inicio se realizó la carga de datos sobre victimas en el datafram df_victimas, el cual se procederá a explorar.

In [ ]:
mf.describir_df(df_victimas)

In [ ]:
# Se visualizan algunos registros
df_victimas.head(3)

### Observaciones iniciales

En total cuenta con **717 registros con 20 campos**:
'ID_hecho', 'FECHA', 'AAAA', 'MM', 'DD', 'ROL', 'VICTIMA', 'SEXO', 'EDAD', 'FECHA_FALLECIMIENTO'.

Entre los mismos encontramos variables temporales, y variables asociadas a la víctima:
- **Información temporal**: campos 'FECHA', 'AAAA', 'MM', 'DD'.
- **Información sobre la víctima**: 'ROL', 'VICTIMA', 'SEXO', 'EDAD', 'FECHA_FALLECIMIENTO'.

Se encuentra el campo **'ID_hecho'**, que se debería corresponder con los identificicadores únicos de 'df_hechos'. Sin embargo, se observa que en este caso no son únicos en su totalidad. Esto seguramente se debe a que en algunos casos un hecho tuvo más de una víctima.

No se registran valores nulos.

Nuevamente se identifican campos con distintos tipos de datos en sus valores, 'EDAD' y 'FECHA_FALLECIMIENTO'.

Como se hizo para el dataframe de hechos, se procede a estandarizar los nombes de los campos.

In [ ]:
print("Campos originales:\n", df_victimas.columns, "\n")
'ID_hecho', 'FECHA', 'AAAA', 'MM', 'DD', 'ROL', 'VICTIMA', 'SEXO', 'EDAD', 'FECHA_FALLECIMIENTO'
# Reemplazar campos específicos
reemplazos = {
    'ID_hecho': 'Id_hecho',
    'FECHA': 'Fecha',
    'AAAA': 'Año',
    'MM': 'Mes',
    'DD': 'Dia',
    'ROL': 'Rol',
    'VICTIMA': 'Victima',
    'SEXO': 'Sexo',
    'EDAD' : 'Edad',
    'FECHA_FALLECIMIENTO' : 'Fecha_fallecimiento'
}

df_victimas.columns = df_victimas.columns.to_series().replace(reemplazos)

print("Campos modificados:\n", df_victimas.columns)

## Tipo de datos

Se exploran los valores de 'Edad'.

In [ ]:
mf.tipo_datos(df_victimas, ['Edad'])

In [ ]:
# Se exploran los valores de tipo texto contenidos en el campo 'Edad'
print("Valores de tipo texto en campo 'Edad':", (df_victimas['Edad'][df_victimas['Edad'].apply(lambda x: isinstance(x, str))]).unique())

En el campo de edad encontramos 53 registros con valor 'SD'.

Se procederá a imputar el valor más frecuente de forma de completar los datos, sin afectar significativamente el análisis estadístico.

In [ ]:
# Reemplazar 'SD' por NaN en la columna 'Edad' del DataFrame df_victimas
df_victimas['Edad'] = df_victimas['Edad'].replace('SD', np.nan)

# Buscar edad más frecuente
edad_mas_frecuente = df_victimas['Edad'].mode()[0]

# Completar los NaN en la columna 'Edad' con la edad más frecuente
df_victimas['Edad'] = df_victimas['Edad'].fillna(edad_mas_frecuente)

In [ ]:
df_victimas['Edad'] = df_victimas['Edad'].astype(int)
mf.tipo_datos(df_victimas, ['Edad'])

### Duplicados

Se analizan valores duplicados de 'Id_hechos'.

In [ ]:
# Se observan valores de 'Id_hecho' duplicados
duplicados_id = (mf.duplicados(df_victimas, campos_incluir=['Id_hecho'])).sort_values(by = 'Id_hecho')

duplicados_id

Existen 20 registros donde el identificador se repite, en uno de ellos 3 veces. Estos deberían corresponder con los hechos registrados con más de 1 víctima.

Se verifica si existe algún registro donde todos los datos estén duplicados, sin identificar ninguno.

In [ ]:
mf.duplicados(df_victimas)

Se exploran valores únicos en los campos de tipo categórico,  para verificar que todos los valores estén tipificados según las categorías predefinidas.

In [ ]:
campos = ['Rol', 'Victima', 'Sexo', 'Edad']
mf.valores_unicos(df_victimas, campos)

No se encuentra ningún valor no típificado, tampoco outliers en la edad.



### Variables temporales

Los datos de 'Fecha', 'Año', 'Mes' y 'Dia' ya se encuentran en 'df_hechos', por lo cual pueden ser eliminados de este conjunto de datos.

El campo 'Fecha_fallecimiento' para los objetivos de nuestro trabajo no resulta de relevancia, con lo cual tambien se lo eliminará.

In [ ]:
# Variables temporales
campos = ['Fecha', 'Año', 'Mes', 'Dia', 'Fecha_fallecimiento']

# Eliminar las columnas especificadas
df_victimas = df_victimas.drop(['Fecha', 'Año', 'Mes', 'Dia', 'Fecha_fallecimiento'], axis=1)

In [ ]:
df_victimas.head(1)

Los campos temporales no parecen tener datos incorrectos.

### Registros sin datos ('SD')

Se exploran los registros sin datos para determinar cómo tratarlos.

In [ ]:
mf.buscar_valor(df_victimas, valores = 'SD')

Se imputaran los valores más frecuentes para 'Rol', 'Victima' y 'Sexo'.

In [ ]:
mf.buscar_moda(df_victimas, campos=['Rol', 'Victima', 'Sexo'])

In [ ]:
mf.imputar_valores(df_victimas, valor_a_reemplazar='SD', valor_a_imputar='CONDUCTOR', campos = ['Rol'])
mf.imputar_valores(df_victimas, valor_a_reemplazar='SD', valor_a_imputar='MOTO', campos = ['Victima'])
mf.imputar_valores(df_victimas, valor_a_reemplazar='SD', valor_a_imputar='MASCULINO', campos = ['Sexo'])

Se verifica que no haya quedado información como 'SD'.

In [ ]:
mf.buscar_valor(df_victimas, valores = 'SD')

Considerando que no se requiere mayor exploración sobre esta información, se procede a unificarla junto a la de hechos analizada anteriormente, de forma tal de poder realizar un análisis integrado de los datos.

## Unificación de datos

In [ ]:
# Fusionar los DataFrames en función de la columna 'Id_hecho'
df_unificado = pd.merge(df_hechos, df_victimas, on='Id_hecho')
df_unificado.head(1)

En ambos dataframe se coservó la columna 'Victima', con lo cual ahora se encuentra duplicada.

Se busca si existen valores distintos para las mismas.

In [ ]:
df = df_unificado[df_unificado['Victima_x'] != df_unificado['Victima_y']]
campos = ['Victima_x', 'Victima_y']
df[campos]

Efectivamente se encuentran distintos valores, observando que la información correspondiente al dataframe de victimas cuenta con mayor detalle.

Se conservan los valor de la segunda.

In [ ]:
# Eliminar las columnas especificadas
df_unificado = df_unificado.drop(['Victima_x'], axis=1)

df_unificado.rename(columns={'Victima_y': 'Victima'}, inplace=True)

In [ ]:
mf.describir_df(df_unificado)

## Semana / Fin de Semana
Antes de proseguir se agrega información sobre el día de la semana (nombre) en que ocurrió el hecho y si ese día corresponde o no a fin de semana.

In [ ]:
df555 = df_unificado

In [ ]:
# Se extrae el día de la semana (0 = lunes, 6 = domingo)
df_unificado['Nro_dia_semana'] = df_unificado['Fecha'].dt.dayofweek
df_unificado['Nro_dia_semana'] = df_unificado['Nro_dia_semana'] + 1

# Mapear los valores numéricos a nombres de los días
dias = {1: 'Lunes', 2: 'Martes', 3: 'Miércoles', 4: 'Jueves', 5: 'Viernes', 6: 'Sábado', 7: 'Domingo'}

df_unificado['Dia_semana'] = df_unificado['Nro_dia_semana'].map(dias)

# Crear una nueva columna 'Fin_semana' en base a 'Dia_semana'
dias_fin_semana = ['Sábado', 'Domingo']
df_unificado['Fin_semana'] = df_unificado['Dia_semana'].apply(lambda x: 'SI' if x in dias_fin_semana else 'NO')

df_unificado.tail(3)

## Rango horario

Se genera un nuevo campo, 'Rango_horario', distinguiendo entre rangos de 3 horas, partiendo desde las 0 hs.

In [ ]:
# Definir los límites de los rangos horarios
limites = [0, 3, 6, 9, 12, 15, 18, 21, 24]

# Definir las etiquetas para cada rango
etiquetas = ['0-2.59', '3-5.59', '6-8.59', '9-11.59', '12-14.59', '15-17.59', '18-21.59', '22-23.59']

# Crear la nueva columna 'Rango_horario'
df_unificado['Rango_horario'] = pd.cut(df_unificado['Franja_horaria'], bins=limites, labels=etiquetas, include_lowest=True, right=False)

# Mostrar el DataFrame resultante
df_unificado[['Franja_horaria', 'Rango_horario']].head()


In [ ]:
df_unificado.head(1)

## Exploración

En este punto del trabajo se comienza con la exploración de relaciones.

- Cantidad de víctimas en el tiempo.
- Cantidad de víctimas por Comuna
- Cantidad de víctimas por tipo de calle
- 
- 
- 




### Cantidad de víctimas por año

Se explora la cantidad de víctimas por año, agrupando también por:
- Mes
- Dia
- Hora
- Semana / Fin de semana
- Hora, Semana / Fin de Semana
- Rango Horario, Semana / Fin de Semana

Se agrupan los datos por distintas variables de tiempo, que se analizarán utilizando las agrupaciones necesarias.

En principio tomaremos los datos de: 'Fecha', 'Año', 'Mes', 'Dia_semana','Franja_horaria'.

In [ ]:
# Agrupar por Fecha, Año, Mes y Franja_horaria y sumar la cantidad de víctimas
victimas_tiempo = df_unificado.groupby(['Fecha', 'Año', 'Mes', 'Dia_semana', 'Fin_semana','Franja_horaria']).agg({'Cantidad_victimas': 'sum'}).sort_values(by=['Franja_horaria', 'Fecha'], ascending=[True, True]).reset_index()

victimas_tiempo.head(3)

#### Víctimas por Año

Se explora el total de víctimas para cada año.

In [ ]:
# Agrupar por Año
victimas_anual = df_unificado.groupby(['Año']).agg({'Cantidad_victimas': 'sum'}).reset_index()

# Configuración de estilo de seaborn
sns.set(style="whitegrid")

# Graficar la cantidad total de víctimas por año como un gráfico de barras
plt.figure(figsize=(8, 4))
sns.barplot(x='Año', y='Cantidad_victimas', data=victimas_anual)
plt.title('Cantidad Total de Víctimas por Año')
plt.xlabel('Año')
plt.ylabel('Cantidad Total de Víctimas')
plt.show()

Se observa una caída notable a partir de 2019 en la cantidad total de víctimas.

#### Victimas por Año y Mes

Se explora el total de víctimas para cada año por mes.

In [ ]:
# Agrupar por Año, Mes
victimas_mensual = df_unificado.groupby(['Año', 'Mes']).agg({'Cantidad_victimas': 'sum'}).reset_index()

# Configurar el estilo de seaborn
sns.set(style="whitegrid")

# Graficar la evolución mensual de la cantidad de víctimas para cada año
g = sns.FacetGrid(victimas_mensual, col="Año", col_wrap=3, height=2, aspect=2, sharey=False)
g.map(sns.lineplot, "Mes", "Cantidad_victimas", marker='o', palette="viridis")
g.set_titles(col_template="{col_name}", fontsize=12)
g.set_axis_labels("Mes", "Cantidad Víctimas")
g.set(xticks=range(1, 13))  # Establecer las etiquetas del eje x para cada mes
g.add_legend()
plt.tight_layout()
plt.show()

A simple vista no parece haber una tendencia en común a todos los años.

En 2020 se observan valores bajos, con un alza marcada para el mes de diciembre. Esto probablemente este asociado al contexto del momento, donde se transitaron los meses de aislamiento preventivo debido a la crisis de COVID.

Más allá de eso, cada año presenta tendencias diferentes, con valores máximos en distintos meses.

#### Víctimas por Año y Dia

En este punto se observa distribución por año según día de la semana.

In [ ]:
# Agrupar por Año, Día
victimas_dia_semana = df_unificado.groupby(['Año', 'Dia_semana', 'Nro_dia_semana']).agg({'Cantidad_victimas': 'sum'}).sort_values(by=['Año', 'Nro_dia_semana'], ascending=[True, True]).reset_index()
victimas_dia_semana

# Configurar el estilo de seaborn
sns.set(style="whitegrid")

# Graficar la evolución mensual de la cantidad de víctimas para cada año
g = sns.FacetGrid(victimas_dia_semana, col="Año", col_wrap=2, height=2, aspect=3, sharey=True)
g.map(sns.barplot, "Dia_semana", "Cantidad_victimas", palette="viridis")
g.set_titles(col_template="{col_name}", fontsize=12)
g.set_axis_labels("Día", "Cantidad Víctimas")
g.set(xticks=range(0, 7))  # Establecer las etiquetas del eje x para cada día
g.add_legend()
plt.tight_layout()
plt.show()

No se observan tendencias claras, los días con mayor cantidad de víctimas son variables según el año.

#### Victimas por Año y Hora

In [ ]:
# Agrupar por Año y Horario
victimas_hora = df_unificado.groupby(['Año', 'Franja_horaria']).agg({'Cantidad_victimas': 'sum'}).reset_index()
victimas_hora.head(3)

In [ ]:
# Configurar el estilo de seaborn
sns.set(style="whitegrid")

# Graficar la evolución mensual de la cantidad de víctimas para cada año
g = sns.FacetGrid(victimas_hora, col="Año", col_wrap=2, height=2, aspect=5, sharey=False)
g.map(sns.lineplot, "Franja_horaria", "Cantidad_victimas", marker='o', palette="viridis")
g.set_titles(col_template="{col_name}", fontsize=12)
g.set_axis_labels("Rango Horario", "Cantidad Víctimas")
g.set(xticks=range(0, 24))  # Establecer las etiquetas del eje x para cada mes
g.add_legend()
plt.tight_layout()
plt.show()

Se pueden observar una tendencia en común.  A partir de las 2 am hasta 7 am se observa una tendencia creciente, alcanzando picos aproximadamente entre 5 y 7 am.

Se explora si existen diferencias en los rangos horarios según sea fin de semana o no.

#### Víctimas por año, Semana / Fin de semana

In [ ]:
# Agrupar
victimas_semana = victimas_tiempo.groupby(['Dia_semana', 'Fin_semana']).agg({'Cantidad_victimas': 'sum'}).sort_values(by='Cantidad_victimas', ascending = False).reset_index()
victimas_semana

In [ ]:
# Filtrar DataFrame para días de semana y fines de semana
semana = victimas_semana[victimas_semana['Fin_semana'] == 'NO']
fin_de_semana = victimas_semana[victimas_semana['Fin_semana'] == 'SI']

# Calcular el promedio para días de semana y fines de semana
promedio_semana = semana['Cantidad_victimas'].mean()
promedio_fin_de_semana = fin_de_semana['Cantidad_victimas'].mean()

# Mostrar resultados
print(f'Promedio de víctimas para días de semana: {promedio_semana}')
print(f'Promedio de víctimas para fines de semana: {promedio_fin_de_semana}')


Se este análisis se puede observar que el promedio de víctimas según sea o no fin de semana presenta diferencias significativas, siendo mas de un 10% mayor los fines de semana.

Por otro lado, si se observan los valores de viernes y lunes (el día previo y posterior al fin de semana) los valores son mayores que los días entre semana (martes a jueves).

#### Victimas por Año, Hora, Semana / Fin de Semana

En este punto se procura explorar tendencias diferenciando si es día de semana o fin de semana distribuído por horario.

In [ ]:
# Agrupar por Año, Franja Horaria, Fin de Semana
victimas_hora_dia = df_unificado.groupby(['Año', 'Franja_horaria', 'Fin_semana']).agg({'Cantidad_victimas': 'sum'}).reset_index()

# Configurar el estilo de Matplotlib
plt.style.use('seaborn-whitegrid')

# Crear subgráficos por año
fig, axes = plt.subplots(nrows=6, ncols=1, figsize=(12, 12), sharey=True)

# Iterar sobre cada año
for i, year in enumerate(range(2016, 2022)):
    # Filtrar datos por año
    subset = victimas_hora_dia[victimas_hora_dia['Año'] == year]

    # Dividir datos por Fin_semana
    weekend_data = subset[subset['Fin_semana'] == 'SI']
    weekday_data = subset[subset['Fin_semana'] == 'NO']

    # Graficar para Fin_semana
    axes[i].plot(weekend_data['Franja_horaria'], weekend_data['Cantidad_victimas'], label='Fin_semana = SI', marker='o')

    # Graficar para NO Fin_semana
    axes[i].plot(weekday_data['Franja_horaria'], weekday_data['Cantidad_victimas'], label='Fin_semana = NO', marker='o')

    # Configurar etiquetas y leyenda
    axes[i].set_title(f'Año {year}')
    axes[i].set_xlabel('Hora')
    axes[i].set_ylabel('Cantidad Víctimas')
    axes[i].set_xticks(range(24))
    axes[i].legend()

# Ajustar diseño y mostrar
plt.tight_layout()
plt.show()


Con líneas azules se puede observar la distribución horaria para fines de semana y con naranja para los días de semana.

- Los picos para fines de semana se dan entre 4 y 7 am.
- La tendencia para días de semana es más variable según el año, observando picos en distintos horarios.

#### Victimas por Año, Rango Horario, Semana / Fin de Semana

Se observan tendencias por los rangos horarios agrupados de 3 horas, distinguiendo semana y fin de semana.

In [ ]:
# Agrupar por Año, Rango Horario, Fin de Semana
victimas_rango_horario = df_unificado.groupby(['Año', 'Rango_horario', 'Fin_semana']).agg({'Cantidad_victimas': 'sum'}).reset_index()

# Configurar el estilo de Matplotlib
plt.style.use('seaborn-whitegrid')

# Crear subgráficos por año
fig, axes = plt.subplots(nrows=6, ncols=1, figsize=(15, 15), sharey=True)

# Iterar sobre cada año
for i, year in enumerate(range(2016, 2022)):
    # Filtrar datos por año
    subset = victimas_rango_horario[victimas_rango_horario['Año'] == year]

    # Dividir datos por Fin_semana
    weekend_data = subset[subset['Fin_semana'] == 'SI']
    weekday_data = subset[subset['Fin_semana'] == 'NO']

    # Graficar para Fin_semana
    axes[i].plot(weekend_data['Rango_horario'], weekend_data['Cantidad_victimas'], label='Fin_semana = SI', marker='o')

    # Graficar para NO Fin_semana
    axes[i].plot(weekday_data['Rango_horario'], weekday_data['Cantidad_victimas'], label='Fin_semana = NO', marker='o')

    # Configurar etiquetas y leyenda
    axes[i].set_title(f'Año {year}')
    axes[i].set_xlabel('Rango Horario')
    axes[i].set_ylabel('Cantidad Víctimas')
    axes[i].set_xticks(range(8))
    axes[i].legend()

# Ajustar diseño y mostrar
plt.tight_layout()
plt.show()

A partir de lo visualizado en los gráficos presentados, se manifiesta más claramente una tendencia para los fines de semana con picos en los rangos 3-5.59 hs y 6-8.59 hs (entre 3 y 9 de la mañana).

Para días de semana, como se dijo, los datos están más dispersos. La mayoría de los picos ocurren entre 9 y 18 horas.

## Victimas agrupadas por variables espaciales

Se exlora información vinculada a 'Tipo_calle', 'Cruce' y 'Comuna'.

In [ ]:
# Agrupar por 'Tipo_calle', 'Cruce', 'Comuna'
victimas_espacio = df_unificado.groupby(['Tipo_calle', 'Cruce', 'Comuna']).agg({'Cantidad_victimas': 'sum'}).reset_index()
victimas_espacio

#### Victimas por Comuna

Se observan valores para 'Comuna' con 0, que serán excluidos del análisis al discriminar valores por Comuna.

In [ ]:
# Se descartan registros con 'Comuna' igual a 0
victimas_comuna = victimas_espacio[victimas_espacio['Comuna'] != 0].groupby(['Comuna']).agg({'Cantidad_victimas': 'sum'}).sort_values(by='Cantidad_victimas').reset_index()

# Graficar la cantidad de víctimas por Comuna
plt.figure(figsize=(8, 4))
sns.barplot(x='Comuna', y='Cantidad_victimas', data=victimas_comuna, palette='viridis')
plt.title('Cantidad de Víctimas por Comuna')
plt.xlabel('Comuna')
plt.ylabel('Cantidad de Víctimas')
plt.show()


Las comunas 1, 4, 7, 8 y 9 son las que más víctimas registran.

#### Victimas por Tipo de Calle

In [ ]:
# Se agrupa
victimas_calle = victimas_espacio.groupby(['Tipo_calle']).agg({'Cantidad_victimas': 'sum'}).sort_values(by='Cantidad_victimas', ascending=False).reset_index()

# Graficar la cantidad de víctimas por Comuna
plt.figure(figsize=(8, 4))
sns.barplot(x='Tipo_calle', y='Cantidad_victimas', data=victimas_calle, palette='viridis')
plt.title('Cantidad de Víctimas por Tipo de Calle')
plt.xlabel('Tipo de Calle')
plt.ylabel('Cantidad de Víctimas')
plt.show()

Se observa que las avenidas registran con una diferencia importante los mayores registros de víctimas.

#### Victimas por Cruce (SI o NO)

In [ ]:
# Se agrupa
victimas_cruce = victimas_espacio.groupby(['Cruce']).agg({'Cantidad_victimas': 'sum'}).sort_values(by='Cantidad_victimas', ascending=False).reset_index()

# Graficar la cantidad de víctimas por Comuna
plt.figure(figsize=(8, 4))
sns.barplot(x='Cruce', y='Cantidad_victimas', data=victimas_cruce, palette='viridis')
plt.title('Cantidad de Víctimas por Cruce')
plt.xlabel('Cruce')
plt.ylabel('Cantidad de Víctimas')
plt.show()

Más del 70% de víctimas se producen en cruces.

#### Victimas por Tipo de Calle, Cruce

In [ ]:
# Se agrupa
victimas_calle = victimas_espacio.groupby(['Tipo_calle', 'Cruce']).agg({'Cantidad_victimas': 'sum'}).sort_values(by='Cantidad_victimas', ascending=False).reset_index()

# Graficar la cantidad de víctimas por Tipo de Calle y Cruce
plt.figure(figsize=(8, 3))
sns.barplot(x='Tipo_calle', y='Cantidad_victimas', hue='Cruce', data=victimas_calle, palette='viridis')
plt.title('Cantidad de Víctimas por Tipo de Calle y Cruce')
plt.xlabel('Tipo de Calle')
plt.ylabel('Cantidad de Víctimas')
plt.show()


Se observá una concentración de más del 50% de accidentes en avenidas con cruce.

## Victimas según participantes

Se exploran relaciones entre 'Acusado', 'Rol' y 'Victima'.

In [ ]:
# Agrupar por 'Acusado', 'Rol', 'Victima'
victimas_participantes = df_unificado.groupby(['Acusado', 'Victima', 'Rol']).agg({'Cantidad_victimas': 'sum'}).sort_values(by='Cantidad_victimas', ascending = False).reset_index()
victimas_participantes.head(3)

#### Víctimas por Vehículo de la Víctima

In [ ]:
# Agrupar por 'Vehiculo'
victimas_vehiculo = victimas_participantes.groupby(['Victima']).agg({'Cantidad_victimas': 'sum'}).sort_values(by='Cantidad_victimas', ascending = False).reset_index()

# Graficar la cantidad de víctimas por Tipo de Calle y Cruce
plt.figure(figsize=(10, 6))
sns.barplot(x='Victima', y='Cantidad_victimas', data=victimas_vehiculo, palette='viridis')
plt.title('Cantidad de Víctimas por Vehiculo')
plt.xlabel('Vehiculo')
plt.ylabel('Cantidad de Víctimas')
plt.show()

Se observa que por amplia diferencia la mayor cantidad de víctimas utilizan moto como medio de transporte o bien son peatones.

#### Víctimas por Acusado

In [ ]:
# Agrupar por 'Acusado'
victimas_acusado = victimas_participantes.groupby(['Acusado']).agg({'Cantidad_victimas': 'sum'}).sort_values(by='Cantidad_victimas', ascending = False).reset_index()

# Graficar la cantidad de víctimas por Tipo de Calle y Cruce
plt.figure(figsize=(10, 6))
sns.barplot(x='Acusado', y='Cantidad_victimas', data=victimas_acusado, palette='viridis')
plt.title('Cantidad de Víctimas por Acusado')
plt.xlabel('Acusado')
plt.ylabel('Cantidad de Víctimas')
plt.show()

Más del 70% de acusados se encuentran entre 'AUTO', 'PASAJEROS' y 'CARGAS'.

#### Víctimas por Rol

In [ ]:
# Agrupar por 'Vehiculo'
victimas_rol = victimas_participantes.groupby(['Rol']).agg({'Cantidad_victimas': 'sum'}).sort_values(by='Cantidad_victimas', ascending = False).reset_index()

# Graficar la cantidad de víctimas por Tipo de Calle y Cruce
plt.figure(figsize=(8, 4))
sns.barplot(x='Rol', y='Cantidad_victimas', data=victimas_rol, palette='viridis')
plt.title('Cantidad de Víctimas por Rol')
plt.xlabel('Rol')
plt.ylabel('Cantidad de Víctimas')
plt.show()

La mayor cantidad de víctimas tienen un rol de conductor del vehículo o peatones.

#### Víctimas por Acusado y Vehículo de la víctima

In [ ]:
print(victimas_participantes.head(10))

# Graficar la cantidad de víctimas por Acusado y Victima
plt.figure(figsize=(8, 4))
sns.barplot(x='Acusado', y='Cantidad_victimas', hue='Victima', data=victimas_participantes.head(10), palette='viridis')
plt.title('Cantidad de Víctimas por Acusado')
plt.xlabel('Acusado')
plt.ylabel('Cantidad de Víctimas')
plt.show()

En este gráfico se puede observar el tipo de víctimas agrupadas por el acusado. 

En el mismo se representan solo las 10 agrupaciones con mayor cantidad de víctimas. El acusado 'PASAJEROS' con víctima 'PEATON' es donde se registra la mayor cantidad.

### Víctimas por Sexo

In [ ]:
# Agrupar por Sexo y sumar la cantidad de víctimas
victimas_por_sexo = df_unificado.groupby('Sexo')['Cantidad_victimas'].agg('sum').reset_index()

# Configurar el estilo de seaborn
sns.set(style="whitegrid")

# Graficar la cantidad de víctimas por Sexo
plt.figure(figsize=(8, 5))
sns.barplot(x='Sexo', y='Cantidad_victimas', data=victimas_por_sexo, palette='viridis')
plt.title('Cantidad de Víctimas por Sexo')
plt.xlabel('Sexo')
plt.ylabel('Cantidad de Víctimas')
plt.show()

Se observa una preponderancia importante de víctimas de sexo masculino.

### Víctimas por Edades

In [ ]:
plt.figure(figsize=(10, 6))
sns.histplot(df_unificado['Edad'], bins=20, kde=True, palette="viridis")
plt.title('Distribución de Edades')
plt.xlabel('Edad')
plt.ylabel('Frecuencia')
plt.show()

Entre los 20 y 40 años se encuentra la mayor cantidad de víctimas.

## Se guarda el Dataset con las modificaciones realizadas

In [ ]:
df_unificado.to_csv('Datasets/homicidios_unificado.csv', index=False, encoding='utf-8')